In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")
!ls

 Complete.pdf				  PDF_Folder_Biomedical
'covid_question_answer_dataset (1).csv'   PDF_Folder_Financial
 covid_question_answer_dataset.csv	  PDF_Folder_Scientific
 CrossDomainAnalysis.ipynb		 ' QA_system_for_BioMedical_data.ipynb'
 DistilBERT.ipynb			  QA_system_for_scientific_data.ipynb
 EvaluationScript_financial.ipynb	  qa-with-squad-2.0-dataset-and-bert-base.ipynb
 FinBERT.ipynb				  question_answer_dataset_financial.csv
'FineTuningDistillBert_SQAUD .ipynb'	  scientific_question_answer_dataset.csv
 Make_dataset_from_PDFs.ipynb		  Untitled0.ipynb
 my_awesome_qa_model			  Untitled1.ipynb


In [ ]:
!pip install datasets evaluate

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

{'id': '57340aae4776f4190066178f',
 'title': 'Genocide',
 'context': 'Slobodan Milošević, as the former President of Serbia and of Yugoslavia, was the most senior political figure to stand trial at the ICTY. He died on 11 March 2006 during his trial where he was accused of genocide or complicity in genocide in territories within Bosnia and Herzegovina, so no verdict was returned. In 1995, the ICTY issued a warrant for the arrest of Bosnian Serbs Radovan Karadžić and Ratko Mladić on several charges including genocide. On 21 July 2008, Karadžić was arrested in Belgrade, and he is currently in The Hague on trial accused of genocide among other crimes. Ratko Mladić was arrested on 26 May 2011 by Serbian special police in Lazarevo, Serbia. Karadzic was convicted of ten of the eleven charges laid against him and sentenced to 40 years in prison on March 24 2016.',
 'question': 'Which former president was by far the most senior politician to be accused of genocidal crimes by the ICTY?',
 'answ

In [ ]:
squad["test"][0]

{'id': '56cff179234ae51400d9c133',
 'title': 'Frédéric_Chopin',
 'context': 'Chopin\'s original publishers included Maurice Schlesinger and Camille Pleyel. His works soon began to appear in popular 19th-century piano anthologies. The first collected edition was by Breitkopf & Härtel (1878–1902). Among modern scholarly editions of Chopin\'s works are the version under the name of Paderewski published between 1937 and 1966 and the more recent Polish "National Edition", edited by Jan Ekier, both of which contain detailed explanations and discussions regarding choices and sources.',
 'question': 'Who edited the Polish "National Edition" of Chopin\'s works?',
 'answers': {'text': ['Jan Ekier'], 'answer_start': [403]}}

In [ ]:
!pip install transformers -U

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
 training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.728600,2.778563
2,2.300600,2.126101
3,1.840000,2.011740


TrainOutput(global_step=1500, training_loss=2.623054972330729, metrics={'train_runtime': 620.3921, 'train_samples_per_second': 19.343, 'train_steps_per_second': 2.418, 'total_flos': 1175877900288000.0, 'train_loss': 2.623054972330729, 'epoch': 3.0})

In [ ]:
from transformers import TrainingArguments

In [ ]:
question = "what are different search engines?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [ ]:
pwd

'/content/gdrive/MyDrive/Colab Notebooks'

In [ ]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model="/content/gdrive/MyDrive/Colab Notebooks/my_awesome_qa_model/checkpoint-500")
question_answerer(question=question, context=context)

{'score': 0.25022581219673157, 'start': 10, 'end': 21, 'answer': '176 billion'}

In [ ]:
#Mannualt way (Alternative)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/gdrive/MyDrive/Colab Notebooks/my_awesome_qa_model/checkpoint-500")
inputs = tokenizer(question, context, return_tensors="pt")

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("/content/gdrive/MyDrive/Colab Notebooks/my_awesome_qa_model/checkpoint-500")
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [ ]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'176 billion'

In [ ]:
from sklearn.metrics import f1_score
import re
def answer_question3(question, answer_text, ground_truth):
    # Tokenize and preprocess the ground truth
    ground_truth_processed = re.sub(r' ##', '', ground_truth)

    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]

    # Tokenize and preprocess the predicted answer
    predicted_answer_processed = re.sub(r' ##', '', predicted_answer)
    # Ensure that ground truth and predicted answer have the same length
    ground_truth_processed = ground_truth_processed[:len(predicted_answer_processed)]
    # Calculate F1 score for ground truth and predicted answer
    f1_ground_truth_predicted = f1_score(list(ground_truth_processed), list(predicted_answer_processed), average='weighted')
    return predicted_answer, f1_ground_truth_predicted


In [ ]:
question = "what is Artificial intellegence?"
answer_text = "Artificial intellegence is Simulation of human intellegence."
ground_truth = "Simulation of human intellegence"

In [ ]:
def compute_f1(prediction, ground_truth):
    prediction_tokens = prediction.lower().split()
    ground_truth_tokens = ground_truth.lower().split()
    common_tokens = set(prediction_tokens) & set(ground_truth_tokens)
    if len(common_tokens) == 0:
        return 0.0
    precision = len(common_tokens) / len(prediction_tokens)
    recall = len(common_tokens) / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [ ]:
from sklearn.metrics import f1_score
import re
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

def answer_question3(question, answer_text, ground_truth):
    # Tokenize input
    inputs = tokenizer(question, answer_text, return_tensors="pt", truncation=True)

    # Forward pass through the model
    outputs = model(**inputs)

    # Get the start and end logits from the output
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the answer span
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits) + 1  # Add 1 to include the end token
    answer_span = tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx])
    # Compute F1 score
    f1_value = compute_f1(answer_span, ground_truth)
    return answer_span, f1_value

# Example usage:
question = "What is the capital of France?"
answer_text = "The capital of France is Paris."
ground_truth = "Paris"
ans, f1_value = answer_question3(question, answer_text, ground_truth)
print("Answer:", ans)
print("F1 Score:", f1_value)

Answer: paris
F1 Score: 1.0


In [ ]:
question = "What were Amazon's net sales in the first quarter of 2023?"
answer_text = "SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2023. • Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022. Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the . . . The company debuted additional Amazon Original films and series, including action comedy Shotgun Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive producer Donald Glover."
ground_truth = "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
ans, f1_value = answer_question3(question, answer_text, ground_truth)
print("Answer:", ans)
print("F1 Score:", f1_value)

Answer: $ 127. 4 billion
F1 Score: 0.08333333333333334


In [ ]:
question = "How much did net sales increase compared to the first quarter of 2022??"
answer_text = "SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2023. • Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022. Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the . . . The company debuted additional Amazon Original films and series, including action comedy Shotgun Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive producer Donald Glover."
ground_truth = "Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the quarter, net sales increased 11% compared with the first quarter of 2022."
ans, f1_value = answer_question3(question, answer_text, ground_truth)
print("Answer:", ans)
print("F1 Score:", f1_value)

Answer: $ 127. 4 billion
F1 Score: 0.06451612903225806


In [ ]:
question = "What was the impact of foreign exchange rates on net sales?"
answer_text = "SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2023. • Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022. Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the . . . The company debuted additional Amazon Original films and series, including action comedy Shotgun Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive producer Donald Glover."
ground_truth = "Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the quarter, net sales increased 11% compared with the first quarter of 2022."
ans, f1_value = answer_question3(question, answer_text, ground_truth)
print("Answer:", ans)
print("F1 Score:", f1_value)

Answer: $ 116. 4 billion
F1 Score: 0.06451612903225806


In [ ]:
question = "How did North America segment sales change year-over-year?"
answer_text = "SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2023. • Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022. Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the . . . The company debuted additional Amazon Original films and series, including action comedy Shotgun Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive producer Donald Glover."
ground_truth = "North America segment sales increased 11% year-over-year to $76.9 billion."
ans, f1_value = answer_question3(question, answer_text, ground_truth)
print("Answer:", ans)
print("F1 Score:", f1_value)

Answer: $ 116. 4 billion
F1 Score: 0.0
